Performs inference on the test set.

1: Imports & Constants

In [2]:
import sys
import os

# Add the parent directory to the Python path so that 'src' can be imported
sys.path.append(os.path.abspath(".."))

from src.postprocessing import TestDataset, generate_submission
from src.preprocessing import get_transforms, load_data
from src.model import get_resnet_model
from torch.utils.data import DataLoader
import torch
import pandas as pd

# Constants
TEST_CSV = "../data/soil_classification-2025/test_ids.csv"
TEST_DIR = "../data/soil_classification-2025/test"


2: Load Test Data

In [5]:
# Load test dataframe and label encoder
test_df = pd.read_csv(TEST_CSV)
_, label_encoder, _ = load_data("../data/soil_classification-2025/train_labels.csv")

# Get transforms
_, val_transform = get_transforms()

# Create test dataset and loader
test_dataset = TestDataset(test_df, TEST_DIR, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


 3: Load Trained Model

In [6]:
# Load model and weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = get_resnet_model(num_classes=4)
model.load_state_dict(torch.load("best_soil_model.pth", map_location=device))
model.to(device)


c:\Users\rajan\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\rajan\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

4: Run Inference & Save Submission

In [7]:
# Run inference and save submission
submission_df = generate_submission(model, test_loader, label_encoder, device)
submission_df.to_csv("submission.csv", index=False)
print("✅ Saved to submission.csv")


✅ Saved to submission.csv
